<a href="https://colab.research.google.com/github/Oomycota1/50projects50days/blob/master/Cp_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install packages

!pip install freesasa biopython -q
print("Packages installed successfully.")



Packages installed successfully.


In [ ]:
#@title **Click here to upload PDB file**
#@markdown Please make sure you preprocess PDB files if there are missing residues or missing atoms.
#@markdown You can use PDBFixer from OpenMM @https://github.com/openmm/pdbfixer

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  if fn.endswith('.pdb'):
    pdb_file_name = fn
    print(f"PDB file identified: {pdb_file_name}")
  else:
    print("Warning: Uploaded file does not have a .pdb extension.")
    pdb_file_name = None # or handle the error as needed
# Now you can work with the pdb_file_name variable,
# for example, to read its content
if pdb_file_name:
    with open(pdb_file_name, 'r') as f:
        pdb_content = f.read()
        # process the pdb content here
        print(pdb_content[:100]) #print the first 100 characters
else:
    print("No valid PDB file uploaded. Exiting.")


Saving 4uux.pdb to 4uux.pdb
User uploaded file "4uux.pdb" with length 592353 bytes
PDB file identified: 4uux.pdb
HEADER    BIOSYNTHETIC PROTEIN                    31-JUL-14   4UUX              
TITLE     COMPETENC


In [ ]:
#@title Run dCp predictions
import freesasa
import pandas as pd
import os
import numpy as np
from collections import Counter
import glob
from typing import Dict, Tuple, List, Union

# Constants
class Constants:
    """Container for all constant values used in the calculations."""

    # 7-Parameter Model Coefficients (Test-Optimized)
    COEFF_7PARAM = {
        'intercept': 7.2493,
        'charmm_Polar_dASA_hepta_mean': -0.9322,
        'charmm_nonpolar_dASA_hepta_mean': 2.6754,
        'f_plr': -0.2526,
        'f_aplr': 0.1449,
        'f_crg': -0.0815,
        'Molecular_Weight': 1.6211,
        'charmm_Total_dASA_hepta_mean': 1.5141
    }

    # Amino acid properties
    AA_PROPERTIES = {
        # Mean values
        'avg': {
            'total': {
                'A': 108.45, 'R': 234.20, 'N': 150.83, 'D': 149.03, 'C': 130.55,
                'Q': 176.10, 'E': 181.96, 'G': 78.43, 'H': 186.31, 'I': 175.95,
                'L': 183.18, 'K': 207.09, 'M': 190.33, 'F': 215.13, 'P': 147.19,
                'S': 117.31, 'T': 135.25, 'Y': 226.05, 'W': 249.16, 'V': 153.46
            },
            'polar': {
                'A': 26.38, 'R': 149.14, 'N': 114.82, 'D': 114.41, 'C': 24.57,
                'Q': 112.00, 'E': 125.15, 'G': 34.67, 'H': 85.81, 'I': 20.27,
                'L': 24.23, 'K': 23.87, 'M': 23.85, 'F': 23.09, 'P': 16.68,
                'S': 67.06, 'T': 55.26, 'W': 47.55, 'Y': 74.94, 'V': 20.87
            },
            'nonpolar': {
                'A': 82.07, 'R': 85.06, 'N': 36.02, 'D': 34.62, 'C': 105.99,
                'Q': 64.10, 'E': 56.80, 'G': 43.77, 'H': 100.49, 'I': 155.68,
                'L': 158.95, 'K': 183.22, 'M': 166.47, 'F': 192.05, 'P': 130.51,
                'S': 50.25, 'T': 79.99, 'W': 201.61, 'Y': 151.11, 'V': 132.59
            }
        },
        # 25th percentile values
        '25': {
            'total': {
                'A': 100.00, 'R': 220.00, 'N': 140.00, 'D': 140.00, 'C': 120.00,
                'Q': 160.00, 'E': 170.00, 'G': 70.00, 'H': 175.00, 'I': 165.00,
                'L': 175.00, 'K': 195.00, 'M': 180.00, 'F': 205.00, 'P': 140.00,
                'S': 110.00, 'T': 125.00, 'Y': 215.00, 'W': 235.00, 'V': 145.00
            },
            'polar': {
                'A': 20.00, 'R': 140.00, 'N': 105.00, 'D': 105.00, 'C': 20.00,
                'Q': 100.00, 'E': 115.00, 'G': 30.00, 'H': 80.00, 'I': 15.00,
                'L': 20.00, 'K': 20.00, 'M': 20.00, 'F': 20.00, 'P': 15.00,
                'S': 60.00, 'T': 50.00, 'W': 40.00, 'Y': 70.00, 'V': 15.00
            },
            'nonpolar': {
                'A': 75.00, 'R': 80.00, 'N': 30.00, 'D': 30.00, 'C': 100.00,
                'Q': 60.00, 'E': 50.00, 'G': 40.00, 'H': 95.00, 'I': 145.00,
                'L': 150.00, 'K': 175.00, 'M': 160.00, 'F': 185.00, 'P': 125.00,
                'S': 45.00, 'T': 75.00, 'W': 195.00, 'Y': 145.00, 'V': 125.00
            }
        },
        # 75th percentile values
        '75': {
            'total': {
                'A': 115.00, 'R': 245.00, 'N': 160.00, 'D': 155.00, 'C': 140.00,
                'Q': 185.00, 'E': 190.00, 'G': 85.00, 'H': 195.00, 'I': 185.00,
                'L': 190.00, 'K': 215.00, 'M': 200.00, 'F': 225.00, 'P': 155.00,
                'S': 125.00, 'T': 145.00, 'Y': 235.00, 'W': 260.00, 'V': 160.00
            },
            'polar': {
                'A': 30.00, 'R': 155.00, 'N': 120.00, 'D': 120.00, 'C': 30.00,
                'Q': 120.00, 'E': 130.00, 'G': 40.00, 'H': 90.00, 'I': 25.00,
                'L': 30.00, 'K': 25.00, 'M': 25.00, 'F': 25.00, 'P': 20.00,
                'S': 70.00, 'T': 60.00, 'W': 55.00, 'Y': 80.00, 'V': 25.00
            },
            'nonpolar': {
                'A': 90.00, 'R': 90.00, 'N': 40.00, 'D': 40.00, 'C': 110.00,
                'Q': 70.00, 'E': 60.00, 'G': 45.00, 'H': 105.00, 'I': 165.00,
                'L': 165.00, 'K': 190.00, 'M': 175.00, 'F': 200.00, 'P': 135.00,
                'S': 55.00, 'T': 85.00, 'W': 205.00, 'Y': 155.00, 'V': 140.00
            }
        }
    }

    # Amino acid molecular weights (in Da)
    AA_WEIGHTS = {
        'A': 89.09, 'R': 174.20, 'N': 132.12, 'D': 133.10, 'C': 121.15,
        'Q': 146.15, 'E': 147.13, 'G': 75.07, 'H': 155.16, 'I': 131.17,
        'L': 131.17, 'K': 146.19, 'M': 149.21, 'F': 165.19, 'P': 115.13,
        'S': 105.09, 'T': 119.12, 'Y': 181.19, 'W': 204.23, 'V': 117.15
    }

    # Amino acid classifications
    POLAR_RESIDUES = ['R', 'N', 'D', 'Q', 'E', 'H', 'K', 'S', 'T', 'Y']
    NONPOLAR_RESIDUES = ['A', 'C', 'G', 'I', 'L', 'M', 'F', 'P', 'W', 'V']
    CHARGED_RESIDUES = ['R', 'D', 'E', 'K']

    # Conversion factors
    J_TO_KJ = 1e-3
    J_TO_KCAL = 1 / (4.184 * 1000)
    WATER_MW = 18.01528  # Molecular weight of water for peptide bond calculation

    # 3-letter to 1-letter amino acid code mapping
    AA_CODE_MAPPING = {
        'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C',
        'GLN': 'Q', 'GLU': 'E', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
        'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P',
        'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'
    }


class ProteinAnalyzer:
    """Main class for protein analysis and dCp estimation."""

    def __init__(self):
        self.constants = Constants()

    def get_sequence_from_pdb(self, pdb_file: str) -> str:
        """
        Extract sequence from PDB file.
        First checks for SEQRES records, if not found, uses ATOM records.

        Args:
            pdb_file: Path to the PDB file

        Returns:
            Protein sequence as a string of 1-letter codes
        """
        # First try to get sequence from SEQRES records
        seqres_sequence = self._get_sequence_from_seqres(pdb_file)
        if seqres_sequence:
            return seqres_sequence

        # Fallback to ATOM records if SEQRES not available
        return self._get_sequence_from_atoms(pdb_file)

    def _get_sequence_from_seqres(self, pdb_file: str) -> Union[str, None]:
        """Extract sequence from SEQRES records."""
        try:
            with open(pdb_file, 'r') as file:
                seqres_lines = [line for line in file if line.startswith('SEQRES')]

            if not seqres_lines:
                return None

            # Group by chain
            chains = {}
            for line in seqres_lines:
                chain_id = line[11]
                if chain_id not in chains:
                    chains[chain_id] = []
                # Extract sequence part (positions 19-70)
                seq_part = line[19:70].strip().split()
                chains[chain_id].extend(seq_part)

            # Convert to single sequence (taking first chain)
            if chains:
                first_chain = list(chains.values())[0]
                sequence = ''.join([
                    self.constants.AA_CODE_MAPPING[residue]
                    for residue in first_chain
                    if residue in self.constants.AA_CODE_MAPPING
                ])

                if sequence:
                    print(f"Using sequence from SEQRES: {len(sequence)} residues")
                    return sequence

        except Exception as e:
            print(f"Error reading SEQRES records: {e}")
            return None

    def _get_sequence_from_atoms(self, pdb_file: str) -> str:
        """Extract sequence from ATOM records (CA atoms)."""
        print("SEQRES not found or error occurred, extracting from ATOM records...")
        seen_residues = set()
        sequence = ""

        with open(pdb_file, 'r') as file:
            for line in file:
                if line.startswith('ATOM') and line[12:16].strip() == 'CA':
                    residue_num = int(line[22:26].strip())
                    residue_code = line[17:20].strip()

                    if (residue_num not in seen_residues and
                        residue_code in self.constants.AA_CODE_MAPPING):
                        sequence += self.constants.AA_CODE_MAPPING[residue_code]
                        seen_residues.add(residue_num)

        return sequence

    def calculate_pdb_asa(self, pdb_file: str) -> Tuple[float, float, float]:
        """Calculate ASA for a given PDB file using FreeSASA."""
        try:
            structure = freesasa.Structure(pdb_file)
            result = freesasa.calc(structure)

            native_total_asa = result.totalArea()

            classifier = freesasa.Classifier()
            areas = freesasa.classifyResults(result, structure, classifier)
            native_polar_asa = areas['Polar']
            native_nonpolar_asa = areas['Apolar']

            return native_total_asa, native_polar_asa, native_nonpolar_asa
        except Exception as e:
            print(f"Error processing PDB file {pdb_file}: {e}")
            return 0.0, 0.0, 0.0

    def calculate_sequence_asa(self, sequence: str, percentile_type: str = 'avg') -> Tuple[float, float, float]:
        """
        Calculate ASA for a given protein sequence using values for a specific percentile.

        Args:
            sequence: Protein sequence as 1-letter codes
            percentile_type: '25', 'avg', or '75' for different ASA values

        Returns:
            Tuple of (total_asa, polar_asa, nonpolar_asa)
        """
        if percentile_type not in self.constants.AA_PROPERTIES:
            percentile_type = 'avg'

        properties = self.constants.AA_PROPERTIES[percentile_type]
        total_asa = polar_asa = nonpolar_asa = 0.0

        for residue in sequence.upper():
            if residue in properties['total']:
                total_asa += properties['total'][residue]
                polar_asa += properties['polar'][residue]
                nonpolar_asa += properties['nonpolar'][residue]
            else:
                print(f"Warning: Residue {residue} not recognized.")

        return total_asa, polar_asa, nonpolar_asa

    def calculate_fraction_features(self, sequence: str) -> Tuple[float, float, float]:
        """Calculate fraction of polar, nonpolar, and charged residues."""
        total_len = len(sequence)
        if total_len == 0:
            return 0.0, 0.0, 0.0

        polar_count = sum(1 for aa in sequence if aa in self.constants.POLAR_RESIDUES)
        nonpolar_count = sum(1 for aa in sequence if aa in self.constants.NONPOLAR_RESIDUES)
        charged_count = sum(1 for aa in sequence if aa in self.constants.CHARGED_RESIDUES)

        return (
            polar_count / total_len,
            nonpolar_count / total_len,
            charged_count / total_len
        )

    def calculate_molecular_weight(self, sequence: str) -> float:
        """Calculate molecular weight of the sequence."""
        weight = sum(self.constants.AA_WEIGHTS.get(aa, 0) for aa in sequence)
        # Subtract water molecular weight for peptide bonds
        water_weight = (len(sequence) - 1) * self.constants.WATER_MW
        return weight - water_weight

    def calculate_dcp_7param(
        self,
        polar_dasa: float,
        nonpolar_dasa: float,
        f_plr: float,
        f_aplr: float,
        f_crg: float,
        molecular_weight: float,
        total_dasa: float
    ) -> float:
        """Calculate dCp using the 7-parameter model."""
        return (
            self.constants.COEFF_7PARAM['intercept'] +
            self.constants.COEFF_7PARAM['charmm_Polar_dASA_hepta_mean'] * polar_dasa +
            self.constants.COEFF_7PARAM['charmm_nonpolar_dASA_hepta_mean'] * nonpolar_dasa +
            self.constants.COEFF_7PARAM['f_plr'] * f_plr +
            self.constants.COEFF_7PARAM['f_aplr'] * f_aplr +
            self.constants.COEFF_7PARAM['f_crg'] * f_crg +
            self.constants.COEFF_7PARAM['Molecular_Weight'] * molecular_weight +
            self.constants.COEFF_7PARAM['charmm_Total_dASA_hepta_mean'] * total_dasa
        )

    def estimate_dcp_from_pdb(self, pdb_file: str) -> Dict:
        """
        Estimate dCp from a PDB file using 7-parameter model
        with all three percentiles (25th, mean, 75th).

        Args:
            pdb_file: Path to the PDB file

        Returns:
            Dictionary containing all results and calculations
        """
        # Extract sequence from PDB
        sequence = self.get_sequence_from_pdb(pdb_file)
        if not sequence:
            raise ValueError(f"Could not extract sequence from {pdb_file}")

        # Calculate native ASA from PDB
        native_total_asa, native_polar_asa, native_nonpolar_asa = self.calculate_pdb_asa(pdb_file)

        # Calculate common features for 7-parameter model
        f_plr, f_aplr, f_crg = self.calculate_fraction_features(sequence)
        molecular_weight = self.calculate_molecular_weight(sequence)

        # Store all results
        detailed_results = {}
        all_dcp_estimates = []
        percentiles = ['25', 'avg', '75']

        for pct in percentiles:
            # Calculate unfolded ASA for this percentile
            unfolded_total, unfolded_polar, unfolded_nonpolar = self.calculate_sequence_asa(sequence, pct)

            # Calculate dASA values
            total_dasa = unfolded_total - native_total_asa
            polar_dasa = unfolded_polar - native_polar_asa
            nonpolar_dasa = unfolded_nonpolar - native_nonpolar_asa

            if total_dasa < 0:
                print(f"Warning: Negative Total ASA Change for {pct} percentile ({total_dasa:.2f})")

            # Calculate dCp using 7-parameter model
            dcp_7param = self.calculate_dcp_7param(
                polar_dasa, nonpolar_dasa, f_plr, f_aplr, f_crg,
                molecular_weight, total_dasa
            )

            # Store results for this percentile
            detailed_results[pct] = {
                'unfolded_total_asa': unfolded_total,
                'unfolded_polar_asa': unfolded_polar,
                'unfolded_nonpolar_asa': unfolded_nonpolar,
                'Total_dASA': total_dasa,
                'Polar_dASA': polar_dasa,
                'Nonpolar_dASA': nonpolar_dasa,
                'dCp_7param': dcp_7param
            }

            all_dcp_estimates.append(dcp_7param)

        # Calculate overall statistics
        dcp_avg = np.mean(all_dcp_estimates)
        dcp_std = np.std(all_dcp_estimates, ddof=1)  # Sample standard deviation

        # Convert to other units
        dcp_avg_kJ = dcp_avg * self.constants.J_TO_KJ
        dcp_std_kJ = dcp_std * self.constants.J_TO_KJ
        dcp_avg_kcal = dcp_avg * self.constants.J_TO_KCAL
        dcp_std_kcal = dcp_std * self.constants.J_TO_KCAL

        # Print results
        self._print_results(
            pdb_file, sequence, native_total_asa, native_polar_asa,
            native_nonpolar_asa, f_plr, f_aplr, f_crg, molecular_weight,
            detailed_results, dcp_avg, dcp_std, dcp_avg_kJ, dcp_std_kJ,
            dcp_avg_kcal, dcp_std_kcal
        )

        # Prepare comprehensive data for CSV
        csv_data = self._prepare_csv_data(
            pdb_file, sequence, native_total_asa, native_polar_asa,
            native_nonpolar_asa, f_plr, f_aplr, f_crg, molecular_weight,
            detailed_results, dcp_avg, dcp_std, dcp_avg_kJ, dcp_std_kJ,
            dcp_avg_kcal, dcp_std_kcal
        )

        return {
            'dCp_avg': dcp_avg,
            'dCp_std': dcp_std,
            'dCp_avg_kJ': dcp_avg_kJ,
            'dCp_std_kJ': dcp_std_kJ,
            'dCp_avg_kcal': dcp_avg_kcal,
            'dCp_std_kcal': dcp_std_kcal,
            'detailed_results': detailed_results,
            'all_estimates': all_dcp_estimates,
            'sequence': sequence,
            'csv_data': csv_data,
            'features': {
                'f_plr': f_plr,
                'f_aplr': f_aplr,
                'f_crg': f_crg,
                'molecular_weight': molecular_weight,
                'native_total_asa': native_total_asa,
                'native_polar_asa': native_polar_asa,
                'native_nonpolar_asa': native_nonpolar_asa
            }
        }

    def _print_results(
        self,
        pdb_file: str,
        sequence: str,
        native_total_asa: float,
        native_polar_asa: float,
        native_nonpolar_asa: float,
        f_plr: float,
        f_aplr: float,
        f_crg: float,
        molecular_weight: float,
        detailed_results: Dict,
        dcp_avg: float,
        dcp_std: float,
        dcp_avg_kJ: float,
        dcp_std_kJ: float,
        dcp_avg_kcal: float,
        dcp_std_kcal: float
    ) -> None:
        """Print formatted results to console."""
        print(f"\n=== dCp Analysis for {os.path.basename(pdb_file)} ===")
        print(f"Sequence length: {len(sequence)} residues")
        print(f"Sequence: {sequence[:50]}{'...' if len(sequence) > 50 else ''}")

        print(f"\n--- Native ASA Values ---")
        print(f"Native Total ASA: {native_total_asa:.1f} Å²")
        print(f"Native Polar ASA: {native_polar_asa:.1f} Å²")
        print(f"Native Nonpolar ASA: {native_nonpolar_asa:.1f} Å²")

        print(f"\n--- Results by Percentile ---")
        for pct in ['25', 'avg', '75']:
            results = detailed_results[pct]
            print(f"\n{pct} percentile:")
            print(f"  Unfolded Total ASA: {results['unfolded_total_asa']:.1f} Å²")
            print(f"  Total dASA: {results['Total_dASA']:.1f} Å²")
            print(f"  Polar dASA: {results['Polar_dASA']:.1f} Å²")
            print(f"  Nonpolar dASA: {results['Nonpolar_dASA']:.1f} Å²")
            print(f"  dCp (7-param): {results['dCp_7param']:.1f} J/(mol·K)")

        print(f"\n--- Additional Features (for 7-param model) ---")
        print(f"Fraction polar: {f_plr:.3f}")
        print(f"Fraction nonpolar: {f_aplr:.3f}")
        print(f"Fraction charged: {f_crg:.3f}")
        print(f"Molecular weight: {molecular_weight:.1f} Da")

        print(f"\n--- Final dCp Estimation Results ---")
        print(f"**AVERAGE dCp (7-parameter model): {dcp_avg:.1f} ± {dcp_std:.1f} J/(mol·K)**")
        print(f"                              {dcp_avg_kJ:.3f} ± {dcp_std_kJ:.3f} kJ/(mol·K)")
        print(f"                              {dcp_avg_kcal:.4f} ± {dcp_std_kcal:.4f} kcal/(mol·K)")
        print("="*60)

    def _prepare_csv_data(
        self,
        pdb_file: str,
        sequence: str,
        native_total_asa: float,
        native_polar_asa: float,
        native_nonpolar_asa: float,
        f_plr: float,
        f_aplr: float,
        f_crg: float,
        molecular_weight: float,
        detailed_results: Dict,
        dcp_avg: float,
        dcp_std: float,
        dcp_avg_kJ: float,
        dcp_std_kJ: float,
        dcp_avg_kcal: float,
        dcp_std_kcal: float
    ) -> Dict:
        """Prepare comprehensive data dictionary for CSV output."""
        pdb_basename = os.path.basename(pdb_file)
        return {
            'PDB_ID': pdb_basename,
            'Sequence': sequence,
            'Sequence_Length': len(sequence),
            'Native_Total_ASA': native_total_asa,
            'Native_Polar_ASA': native_polar_asa,
            'Native_Nonpolar_ASA': native_nonpolar_asa,
            'f_polar': f_plr,
            'f_nonpolar': f_aplr,
            'f_charged': f_crg,
            'Molecular_Weight': molecular_weight,
            # 25th percentile results
            'Unfolded_Total_ASA_25pct': detailed_results['25']['unfolded_total_asa'],
            'Total_dASA_25pct': detailed_results['25']['Total_dASA'],
            'Polar_dASA_25pct': detailed_results['25']['Polar_dASA'],
            'Nonpolar_dASA_25pct': detailed_results['25']['Nonpolar_dASA'],
            'dCp_7param_25pct': detailed_results['25']['dCp_7param'],
            # Mean results
            'Unfolded_Total_ASA_mean': detailed_results['avg']['unfolded_total_asa'],
            'Total_dASA_mean': detailed_results['avg']['Total_dASA'],
            'Polar_dASA_mean': detailed_results['avg']['Polar_dASA'],
            'Nonpolar_dASA_mean': detailed_results['avg']['Nonpolar_dASA'],
            'dCp_7param_mean': detailed_results['avg']['dCp_7param'],
            # 75th percentile results
            'Unfolded_Total_ASA_75pct': detailed_results['75']['unfolded_total_asa'],
            'Total_dASA_75pct': detailed_results['75']['Total_dASA'],
            'Polar_dASA_75pct': detailed_results['75']['Polar_dASA'],
            'Nonpolar_dASA_75pct': detailed_results['75']['Nonpolar_dASA'],
            'dCp_7param_75pct': detailed_results['75']['dCp_7param'],
            # Averages and standard deviations
            'dCp_avg_J_mol_K': dcp_avg,
            'dCp_std_J_mol_K': dcp_std,
            # All unit conversions
            'dCp_avg_kJ_mol_K': dcp_avg_kJ,
            'dCp_std_kJ_mol_K': dcp_std_kJ,
            'dCp_avg_kcal_mol_K': dcp_avg_kcal,
            'dCp_std_kcal_mol_K': dcp_std_kcal
        }

    def process_directory(self, content_dir: str = "content") -> Dict:
        """
        Process all PDB files in the specified directory.

        Args:
            content_dir: Path to the directory containing PDB files

        Returns:
            Dictionary containing summary of all processed files
        """
        # Find all PDB files in the content directory
        pdb_patterns = [
            os.path.join(content_dir, "*.pdb"),
            os.path.join(content_dir, "*.PDB")
        ]

        pdb_files = []
        for pattern in pdb_patterns:
            pdb_files.extend(glob.glob(pattern))

        if not pdb_files:
            print(f"No PDB files found in {content_dir} directory")
            return {}

        print(f"Found {len(pdb_files)} PDB files to process:")
        for pdb_file in pdb_files:
            print(f"  - {os.path.basename(pdb_file)}")

        # Process each PDB file
        all_results = []
        summary_data = []

        for i, pdb_file in enumerate(pdb_files, 1):
            print(f"\n{'='*80}")
            print(f"Processing file {i}/{len(pdb_files)}: {os.path.basename(pdb_file)}")
            print(f"{'='*80}")

            try:
                results = self.estimate_dcp_from_pdb(pdb_file)
                all_results.append({
                    'file': pdb_file,
                    'results': results
                })

                # Add to summary
                summary_data.append(results['csv_data'])

            except Exception as e:
                print(f"Error processing {pdb_file}: {e}")
                continue

        # Create combined results CSV if we have data
        if not summary_data:
            return {'processed_files': 0, 'results': [], 'combined_data': []}

        combined_df = pd.DataFrame(summary_data)
        combined_csv = "dCp_analysis_all_pdbs.csv"
        combined_df.to_csv(combined_csv, index=False)
        print(f"\n✓ Combined results saved to: {combined_csv}")

        # Print summary statistics
        self._print_summary_statistics(combined_df)

        return {
            'processed_files': len(summary_data),
            'results': all_results,
            'combined_data': summary_data,
            'combined_csv': combined_csv
        }

    def _print_summary_statistics(self, combined_df: pd.DataFrame) -> None:
        """Print summary statistics for batch processing."""
        print(f"\n{'='*80}")
        print("SUMMARY STATISTICS")
        print(f"{'='*80}")
        print(f"Total files processed: {len(combined_df)}")
        print(f"Average dCp across all proteins: {combined_df['dCp_avg_J_mol_K'].mean():.1f} ± "
              f"{combined_df['dCp_avg_J_mol_K'].std():.1f} J/(mol·K)")
        print(f"                               {combined_df['dCp_avg_kJ_mol_K'].mean():.3f} ± "
              f"{combined_df['dCp_avg_kJ_mol_K'].std():.3f} kJ/(mol·K)")
        print(f"                               {combined_df['dCp_avg_kcal_mol_K'].mean():.4f} ± "
              f"{combined_df['dCp_avg_kcal_mol_K'].std():.4f} kcal/(mol·K)")

        # Show range
        min_dcp = combined_df['dCp_avg_J_mol_K'].min()
        max_dcp = combined_df['dCp_avg_J_mol_K'].max()
        print(f"\nRange: {min_dcp:.1f} to {max_dcp:.1f} J/(mol·K)")
        print(f"Range: {min_dcp * self.constants.J_TO_KJ:.3f} to {max_dcp * self.constants.J_TO_KJ:.3f} kJ/(mol·K)")
        print(f"Range: {min_dcp * self.constants.J_TO_KCAL:.4f} to {max_dcp * self.constants.J_TO_KCAL:.4f} kcal/(mol·K)")

        # Show top 5 highest and lowest dCp values
        sorted_by_dcp = combined_df.sort_values('dCp_avg_J_mol_K')
        print(f"\nTop 5 highest dCp values:")
        for _, row in sorted_by_dcp.tail(5).iterrows():
            print(f"  {row['PDB_ID']}: {row['dCp_avg_J_mol_K']:.1f} J/(mol·K)")

        print(f"\nTop 5 lowest dCp values:")
        for _, row in sorted_by_dcp.head(5).iterrows():
            print(f"  {row['PDB_ID']}: {row['dCp_avg_J_mol_K']:.1f} J/(mol·K)")


def main():
    """Main execution function."""
    print("Starting batch processing of PDB files...")
    analyzer = ProteinAnalyzer()
    final_results = analyzer.process_directory("/content/")

    print(f"\n{'='*80}")
    print("BATCH PROCESSING COMPLETE")
    print(f"{'='*80}")
    print(f"Successfully processed {final_results['processed_files']} PDB files")
    if final_results.get('combined_csv'):
        print(f"Results saved to: {final_results['combined_csv']}")
    print("Done!")


if __name__ == "__main__":
    main()

Starting batch processing of PDB files...
Found 2 PDB files to process:
  - 1bnr_fixed.pdb
  - 4uux.pdb

Processing file 1/2: 1bnr_fixed.pdb
SEQRES not found or error occurred, extracting from ATOM records...

=== dCp Analysis for 1bnr_fixed.pdb ===
Sequence length: 110 residues
Sequence: AQVINTFDGVADYLQTYHKLPDNYITKSEAQALGWVASKGNLADVAPGKS...

--- Native ASA Values ---
Native Total ASA: 6335.0 Å²
Native Polar ASA: 3053.6 Å²
Native Nonpolar ASA: 3281.4 Å²

--- Results by Percentile ---

25 percentile:
  Unfolded Total ASA: 16670.0 Å²
  Total dASA: 10335.0 Å²
  Polar dASA: 2931.4 Å²
  Nonpolar dASA: 7133.6 Å²
  dCp (7-param): 52081.5 J/(mol·K)

avg percentile:
  Unfolded Total ASA: 17791.6 Å²
  Total dASA: 11456.6 Å²
  Polar dASA: 3629.3 Å²
  Nonpolar dASA: 7827.4 Å²
  dCp (7-param): 54985.3 J/(mol·K)

75 percentile:
  Unfolded Total ASA: 18685.0 Å²
  Total dASA: 12350.0 Å²
  Polar dASA: 4131.4 Å²
  Nonpolar dASA: 8418.6 Å²
  dCp (7-param): 57451.6 J/(mol·K)

--- Additional Features (for 